In [1]:
from dotenv import load_dotenv
import cohere
import os
load_dotenv()

co = cohere.Client(os.environ['COHERE_API'])

In [ ]:
input_file = ""
output_file = ""

In [2]:
system_msg = 'Você é um analista financeiro com formação em economia que escreve para um público de investidores geral'


def get_user_prompt(company, fr):
    return f'''### Instrução:
Escreva em Português uma análise sobre a Empresa considerando o Fato Relevante e siga o Formato determinado. A análise deve conter TODOS os elementos especificados no seguinte Formato.

### Formato:
<Título>
<Sentença que Resuma a análise>
<Corpo da análise>
<Considerações Finais>

### Empresa:
{company}

### Fato Relevante:
{fr}

### Resposta: '''

In [ ]:
import pandas as pd

df = pd.read_csv(input_file)

In [4]:
generated_texts = [None] * len(df)

In [5]:
for index, row in df.iterrows():
    try:
        empresa = row['empresa']
        fr_conteudo = row['FR_conteudo']

        instruct = get_user_prompt(empresa, fr_conteudo)

        messages = [
            {"role": "system", "message": system_msg},
        ]
        
        response = co.chat(
            chat_history=messages,
            message=instruct,
            temperature=0.3,
            max_tokens=2000,
            p=0.4
        )

        generated_texts[index] = response.text
    
    except Exception as e:
        print(f"Erro: {e}")
        continue


In [6]:
df.loc[:len(generated_texts)-1, 'generated_text'] = generated_texts

In [ ]:
df.to_csv(output_file, index=False)